In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px #more common and simple use
import plotly.graph_objects as go #work moslty on graph objects (more heavy and dynamic) easier to control
from sklearn.model_selection import train_test_split

In [2]:
air_BNB_Train = pd.read_csv("train_airBNB.csv")

In [3]:
air_BNB_Train.head()

id                            listing_url     scrape_id last_scraped  \
0    761411    https://www.airbnb.com/rooms/761411  2.020000e+13   08/07/2021   
1   5753687   https://www.airbnb.com/rooms/5753687  2.020000e+13   08/06/2021   
2  19807045  https://www.airbnb.com/rooms/19807045  2.020000e+13   08/06/2021   
3   1272351   https://www.airbnb.com/rooms/1272351  2.020000e+13   08/07/2021   
4  19190181  https://www.airbnb.com/rooms/19190181  2.020000e+13   08/06/2021   

                                              name  \
0  30% discount A'dam Apartment - Clean Quiet Safe   
1                   Bright apartment in "de Pijp"!   
2                     Stay in your comfort zone :)   
3                Spacious modern 2 bedr flat 100m2   
4           Apartment in Trendy Amsterdam Old-West   

                                         description  \
0  30% discount due to Covid-19 temporary.<br /><...   
1  The apartment is located in the popular area c...   
2  Perfectly maintained two-bedroom apartment of ...   
3  A large and bright recently built appartment i...   
4                     <b>The space</b><br />a Couple   

                               neighborhood_overview  \
0  The neighborhood is very green, quiet, safe an...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                         picture_url   host_id  \
0  https://a0.muscache.com/pictures/10591508/bf30...   4013546   
1  https://a0.muscache.com/pictures/056a6bcb-5996...  29848509   
2  https://a0.muscache.com/pictures/8f77bce3-b87a...  28218333   
3  https://a0.muscache.com/pictures/20948245/5b79...   1743443   
4  https://a0.muscache.com/pictures/ee6a527c-b855...  15484516   

                                     host_url  ... availability_90  \
0   https://www.airbnb.com/users/show/4013546  ...               8   
1  https://www.airbnb.com/users/show/29848509  ...               0   
2  https://www.airbnb.com/users/show/28218333  ...               0   
3   https://www.airbnb.com/users/show/1743443  ...               0   
4  https://www.airbnb.com/users/show/15484516  ...               0   

  availability_365 calendar_last_scraped number_of_reviews  \
0              167            08/07/2021               254   
1                0            08/06/2021                 3   
2                0            08/06/2021                13   
3                0            08/07/2021                10   
4                0            08/06/2021                16   

  number_of_reviews_ltm number_of_reviews_l30d first_review last_review  \
0                     6                      1   05/08/2014   6/17/2021   
1                     0                      0    3/30/2015   6/27/2016   
2                     0                      0   01/07/2018  04/04/2019   
3                     0                      0   08/05/2019   6/15/2015   
4                     0                      0   10/29/2017   9/21/2018   

  review_scores_rating                   license  
0                 4.75  0363 D4AD DCF3 E72A 56A2  
1                 4.67                       NaN  
2                 4.92                       NaN  
3                 5.00                       NaN  
4                 4.67                       NaN  

[5 rows x 62 columns]

In [4]:
#first drop of unnecessary features for prediction 
with open('featuers_removed_unnecessary.txt','w') as f:
  f.write('The features removed are : scrape_id,listing_url,name, review_scores_rating,last_scraped,picture_url,host_url,calendar_last_scraped,license,first_review,last_review ')
air_BNB_Train.drop(['scrape_id','host_thumbnail_url','host_picture_url','listing_url','host_has_profile_pic','host_since','last_scraped','picture_url','name','host_url','calendar_last_scraped','license','first_review','host_location','last_review','host_name','host_id','review_scores_rating'],axis = 1,inplace = True)

In [5]:
air_BNB_Train.isnull().sum()
#Drop all the featuers that have more than 4000 Null values
air_BNB_Train.dropna(axis=1, thresh=8000, inplace=True)

In [6]:
air_BNB_Train.isnull().sum()

id                             0
description                  166
host_is_superhost             39
host_listings_count           39
host_total_listings_count     39
host_verifications             0
host_identity_verified        39
neighbourhood_cleansed         0
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms_text                13
bedrooms                     635
beds                          67
amenities                      0
price                          0
minimum_nights                 0
maximum_nights                 0
minimum_minimum_nights         3
maximum_minimum_nights         3
minimum_maximum_nights         3
maximum_maximum_nights         3
minimum_nights_avg_ntm         3
maximum_nights_avg_ntm         3
has_availability               0
availability_30                0
availability_60                0
availability_90                0
availabili

In [7]:
#Check how many unique values we have in each column
air_BNB_Train.nunique()

id                           11505
description                  11230
host_is_superhost                2
host_listings_count             37
host_total_listings_count       37
host_verifications             315
host_identity_verified           2
neighbourhood_cleansed          22
latitude                      5096
longitude                     7335
property_type                   63
room_type                        4
accommodates                    16
bathrooms_text                  25
bedrooms                        11
beds                            21
amenities                    10664
price                          474
minimum_nights                  53
maximum_nights                 118
minimum_minimum_nights          53
maximum_minimum_nights          54
minimum_maximum_nights         107
maximum_maximum_nights         109
minimum_nights_avg_ntm         117
maximum_nights_avg_ntm         209
has_availability                 2
availability_30                 31
availability_60     

In [8]:
temp = air_BNB_Train['price'].str.replace('$', '')
p = temp.str.replace(',', '')
p = p.astype(float)
type(p[0])
air_BNB_Train['price'] = p
price = air_BNB_Train['price']

In [9]:
#Delete Columns that require NLP or further Geopy process
air_BNB_Train.drop(['amenities','description','latitude','longitude','bathrooms_text','accommodates','host_verifications'],axis = 1,inplace = True)

In [10]:
# Replace f with 0 and t with 1 
air_BNB_Train.host_is_superhost.replace({'t':1, 'f':0}, inplace=True)
air_BNB_Train.has_availability.replace({'t':1, 'f':0}, inplace=True)
air_BNB_Train.host_identity_verified.replace({'t':1, 'f':0}, inplace=True)

In [11]:
air_BNB_Train = pd.get_dummies(air_BNB_Train, columns=['neighbourhood_cleansed'])
air_BNB_Train = pd.get_dummies(air_BNB_Train, columns=['property_type'])
air_BNB_Train = pd.get_dummies(air_BNB_Train, columns=['room_type'])

In [12]:
corr_df = air_BNB_Train.corr()
corr_df

id  host_is_superhost  host_listings_count  \
id                         1.000000           0.013333             0.031908   
host_is_superhost          0.013333           1.000000             0.002025   
host_listings_count        0.031908           0.002025             1.000000   
host_total_listings_count  0.031908           0.002025             1.000000   
host_identity_verified    -0.019592           0.123792             0.027966   
...                             ...                ...                  ...   
property_type_Yurt         0.011130          -0.003538            -0.000389   
room_type_Entire home/apt -0.114069          -0.239480            -0.004924   
room_type_Hotel room       0.048918           0.090353             0.014712   
room_type_Private room     0.105692           0.227760             0.001778   
room_type_Shared room      0.002991          -0.020041             0.002052   

                           host_total_listings_count  host_identity_verified  \
id                                          0.031908               -0.019592   
host_is_superhost                           0.002025                0.123792   
host_listings_count                         1.000000                0.027966   
host_total_listings_count                   1.000000                0.027966   
host_identity_verified                      0.027966                1.000000   
...                                              ...                     ...   
property_type_Yurt                         -0.000389               -0.013138   
room_type_Entire home/apt                  -0.004924               -0.075032   
room_type_Hotel room                        0.014712                0.047597   
room_type_Private room                      0.001778                0.065880   
room_type_Shared room                       0.002052                0.006099   

                           bedrooms      beds     price  minimum_nights  \
id                        -0.007894 -0.025826  0.019373       -0.028818   
host_is_superhost         -0.056053 -0.012584 -0.013914       -0.006532   
host_listings_count       -0.008096  0.002432  0.010459       -0.002899   
host_total_listings_count -0.008096  0.002432  0.010459       -0.002899   
host_identity_verified    -0.010896  0.008871  0.008932       -0.000286   
...                             ...       ...       ...             ...   
property_type_Yurt        -0.005265  0.001497 -0.005911        0.001124   
room_type_Entire home/apt  0.209111  0.126474  0.136114        0.023214   
room_type_Hotel room      -0.032457 -0.020437 -0.010662       -0.011414   
room_type_Private room    -0.202208 -0.126039 -0.133650       -0.020494   
room_type_Shared room     -0.029827  0.011849 -0.019547       -0.006202   

                           maximum_nights  ...  \
id                              -0.000203  ...   
host_is_superhost               -0.021679  ...   
host_listings_count              0.001652  ...   
host_total_listings_count        0.001652  ...   
host_identity_verified          -0.008336  ...   
...                                   ...  ...   
property_type_Yurt              -0.010368  ...   
room_type_Entire home/apt       -0.017107  ...   
room_type_Hotel room             0.034727  ...   
room_type_Private room           0.010731  ...   
room_type_Shared room           -0.002570  ...   

                           property_type_Shared room in rental unit  \
id                                                        -0.010497   
host_is_superhost                                         -0.015022   
host_listings_count                                       -0.000295   
host_total_listings_count                                 -0.000295   
host_identity_verified                                    -0.004467   
...                                                             ...   
property_type_Yurt                                        -0.000369   
room_type_Entire home/apt                            

In [13]:
fig = go.Figure()
fig.add_traces(go.Heatmap(
    z=corr_df, 
    x=corr_df.columns, 
    y=corr_df.columns,
    zmax=1, 
    zmin=-1
))
fig.update_layout({
    'title':"Features Correlation Heatmap"
})

In [14]:
for i in range(len(corr_df.columns)-1):
  print(corr_df.columns[i], 'is correlated to: ')
  for j in range(len(corr_df.columns)-1):
    if abs(corr_df.iloc[i,j])>=0.85 and i is not j:
      print(corr_df.columns[j], end=', ')
  print('\n----------------------')

id is correlated to: 

----------------------
host_is_superhost is correlated to: 

----------------------
host_listings_count is correlated to: 
host_total_listings_count, 
----------------------
host_total_listings_count is correlated to: 
host_listings_count, 
----------------------
host_identity_verified is correlated to: 

----------------------
bedrooms is correlated to: 

----------------------
beds is correlated to: 

----------------------
price is correlated to: 

----------------------
minimum_nights is correlated to: 
minimum_minimum_nights, maximum_minimum_nights, minimum_nights_avg_ntm, 
----------------------
maximum_nights is correlated to: 

----------------------
minimum_minimum_nights is correlated to: 
minimum_nights, maximum_minimum_nights, minimum_nights_avg_ntm, 
----------------------
maximum_minimum_nights is correlated to: 
minimum_nights, minimum_minimum_nights, minimum_nights_avg_ntm, 
----------------------
minimum_maximum_nights is correlated to: 

-------

In [15]:
#Drop according to correlation.
air_BNB_Train.drop(['minimum_nights','minimum_minimum_nights', 'maximum_minimum_nights','maximum_maximum_nights','host_listings_count','availability_30', 'availability_60','minimum_nights', 'minimum_minimum_nights', 'minimum_nights_avg_ntm'],axis = 1,inplace = True)

In [16]:
air_BNB_Train.host_is_superhost.fillna(value = air_BNB_Train.host_is_superhost.value_counts().index[0], inplace = True)
air_BNB_Train.host_total_listings_count.fillna(value = air_BNB_Train.host_total_listings_count .value_counts().index[0], inplace = True)
air_BNB_Train.host_identity_verified.fillna(value = air_BNB_Train.host_identity_verified.value_counts().index[0], inplace = True)
air_BNB_Train.bedrooms.fillna(value = air_BNB_Train.bedrooms.value_counts().index[0], inplace = True)
air_BNB_Train.beds.fillna(value = air_BNB_Train.beds.value_counts().index[0], inplace = True)
air_BNB_Train.maximum_nights_avg_ntm.fillna(value = air_BNB_Train.maximum_nights_avg_ntm.value_counts().index[0], inplace = True)
air_BNB_Train.minimum_maximum_nights .fillna(value = air_BNB_Train.minimum_maximum_nights .value_counts().index[0], inplace = True)

In [17]:
X = air_BNB_Train.drop('price', axis= 1)
y = price

In [18]:
X.isnull().sum().head(30)

id                                                               0
host_is_superhost                                                0
host_total_listings_count                                        0
host_identity_verified                                           0
bedrooms                                                         0
beds                                                             0
maximum_nights                                                   0
minimum_maximum_nights                                           0
maximum_nights_avg_ntm                                           0
has_availability                                                 0
availability_90                                                  0
availability_365                                                 0
number_of_reviews                                                0
number_of_reviews_ltm                                            0
number_of_reviews_l30d                                        

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
train_df.head(10)

id  host_is_superhost  host_total_listings_count  \
3164   51361172                0.0                        1.0   
683    50150930                0.0                        1.0   
1511   41763717                0.0                        1.0   
7894   17300744                0.0                        3.0   
469     6635669                0.0                        1.0   
7552   21133952                0.0                        1.0   
2109   23855455                0.0                        2.0   
10276  23647399                0.0                        1.0   
7458   34634888                1.0                        1.0   
6388   18966176                0.0                        1.0   

       host_identity_verified  bedrooms  beds  maximum_nights  \
3164                      0.0       1.0   2.0             365   
683                       0.0       2.0   2.0             365   
1511                      1.0       2.0   2.0               8   
7894                      1.0       1.0   2.0               8   
469                       1.0       1.0   1.0            1125   
7552                      0.0       2.0   2.0            1125   
2109                      0.0       1.0   1.0            1125   
10276                     1.0       3.0   3.0              16   
7458                      1.0       1.0   1.0              28   
6388                      0.0       2.0   2.0              58   

       minimum_maximum_nights  maximum_nights_avg_ntm  has_availability  ...  \
3164                   1125.0                  1125.0                 1  ...   
683                     365.0                   365.0                 1  ...   
1511                      8.0                     8.0                 1  ...   
7894                      8.0                     8.0                 1  ...   
469                    1125.0                  1125.0                 1  ...   
7552                   1125.0                  1125.0                 1  ...   
2109                   1125.0                  1125.0                 1  ...   
10276                    16.0                    16.0                 1  ...   
7458                     28.0                    28.0                 1  ...   
6388                     58.0                    58.0                 1  ...   

       property_type_Shared room in residential home  \
3164                                               0   
683                                                0   
1511                                               0   
7894                                               0   
469                                                0   
7552                                               0   
2109                                               0   
10276                                              0   
7458                                               0   
6388                                               0   

       property_type_Tiny house  property_type_Tipi  property_type_Tower  \
3164                          0                   0                    0   
683                           0                   0                    0   
1511                          0                   0                    0   
7894                          0                   0                    0   
469                           0                   0                    0   
7552                          0                   0                    0   
2109                          0                   0                    0   
10276                         0                   0                    0   
7458                          0                   0                    0   
6388                          0                   0                    0   

       property_type_Yurt  room_type_Entire home/apt  room_type_Hotel room  \
3164                    0                          1                     0   
683                     0                          1                     0   
1511        

In [20]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()  # define our model using least square method
lm.fit(X_train,y_train)

#use lm.predict(X_train_scaled) for all the living area data from the train set
fitted_scaled_price = lm.predict(X_train) # returns ndarray

#inverse the scaler transformation
fitted_price = y_train

In [21]:
train_df['residuals'] = fitted_scaled_price -fitted_price

In [22]:
fig= go.Figure()
fig.add_trace(
    go.Scatter(
        x=fitted_scaled_price,
        y=train_df.residuals,
        mode='markers',
        name='train residuals',
        marker_color='blue',
        marker_size=2.5,
        marker_line_width=0,

    )
)
fig.add_trace(
  go.Scatter(
      x=fitted_scaled_price,
      y=train_df.residuals*0,
      mode='lines',
      name='zero line',
      marker_color='black',
      marker_size=1.5,
      marker_line_width=0,

  )
)
fig.update_layout(
    title="Residuals of Predicted prices",
    xaxis_title="Predicted price",
    yaxis_title="Residuals",
    font=dict(
        size=14,
        color="RebeccaPurple"
    )
)
fig.show()

In [23]:
from sklearn import metrics

print("------ TRAIN DATA ------")
print("MSE:",metrics.mean_squared_error(fitted_price, fitted_scaled_price))
print("RMSE:",np.sqrt(metrics.mean_squared_error(fitted_price, fitted_scaled_price)))
print("MAE:",metrics.mean_absolute_error(fitted_price, fitted_scaled_price))

------ TRAIN DATA ------
MSE: 36256.17884471059
RMSE: 190.41055339636662
MAE: 55.85563594904443


In [24]:
air_BNB_test= pd.read_csv("test_airBNB.csv")

In [25]:
air_BNB_test.drop(['scrape_id','host_thumbnail_url','host_picture_url','listing_url','host_has_profile_pic','host_since','last_scraped','picture_url','name','host_url','calendar_last_scraped','license','first_review','host_location','last_review','host_name','host_id'],axis = 1,inplace = True)
air_BNB_test.drop(['amenities','description','latitude','longitude','bathrooms_text','accommodates','host_verifications'],axis = 1,inplace = True)
air_BNB_test.drop(['minimum_nights','minimum_minimum_nights', 'maximum_minimum_nights','maximum_maximum_nights','host_listings_count','availability_30', 'availability_60','minimum_nights', 'minimum_minimum_nights', 'minimum_nights_avg_ntm'],axis = 1,inplace = True)
air_BNB_test.drop(['neighborhood_overview','host_about', 'host_response_time','host_acceptance_rate','host_neighbourhood','neighbourhood', 'neighbourhood_group_cleansed', 'calendar_updated'],axis = 1,inplace = True)

In [26]:
air_BNB_test.dropna(axis=1, thresh=3200, inplace=True)

In [27]:
air_BNB_test.isnull().sum()

id                             0
host_is_superhost             16
host_total_listings_count     16
host_identity_verified        16
neighbourhood_cleansed         0
property_type                  0
room_type                      0
bedrooms                     222
beds                          11
maximum_nights                 0
minimum_maximum_nights         0
maximum_nights_avg_ntm         0
has_availability               0
availability_90                0
availability_365               0
number_of_reviews              0
number_of_reviews_ltm          0
number_of_reviews_l30d         0
dtype: int64

In [28]:
air_BNB_test.host_is_superhost.fillna(value = air_BNB_test.host_is_superhost.value_counts().index[0], inplace = True)
air_BNB_test.host_total_listings_count.fillna(value = air_BNB_test.host_total_listings_count .value_counts().index[0], inplace = True)
air_BNB_test.host_identity_verified.fillna(value = air_BNB_test.host_identity_verified.value_counts().index[0], inplace = True)
air_BNB_test.bedrooms.fillna(value = air_BNB_test.bedrooms.value_counts().index[0], inplace = True)
air_BNB_test.beds.fillna(value = air_BNB_test.beds.value_counts().index[0], inplace = True)
air_BNB_test.maximum_nights_avg_ntm.fillna(value = air_BNB_test.maximum_nights_avg_ntm.value_counts().index[0], inplace = True)
air_BNB_test.minimum_maximum_nights .fillna(value = air_BNB_test.minimum_maximum_nights .value_counts().index[0], inplace = True)

In [29]:
# Replace f with 0 and t with 1 
air_BNB_test.host_is_superhost.replace({'t':1, 'f':0}, inplace=True)
air_BNB_test.has_availability.replace({'t':1, 'f':0}, inplace=True)
air_BNB_test.host_identity_verified.replace({'t':1, 'f':0}, inplace=True)

In [30]:
air_BNB_test = pd.get_dummies(air_BNB_test, columns=['neighbourhood_cleansed'])
air_BNB_test = pd.get_dummies(air_BNB_test, columns=['property_type'])
air_BNB_test = pd.get_dummies(air_BNB_test, columns=['room_type'])

In [31]:
test_columns = air_BNB_test.columns
train_columns = air_BNB_Train.columns
m = []
for i in train_columns:
  if(i not in test_columns):
    m.append(i)
for i in m:
  if(i != 'price'):
    air_BNB_test[i] = 0

In [32]:
test_columns = air_BNB_test.columns
train_columns = air_BNB_Train.columns
m = []
for i in test_columns:
  if(i not in train_columns):
    m.append(i)
for i in m:
  air_BNB_test.drop([i], axis =1, inplace = True)

In [33]:
fitted_scaled_price_test = lm.predict(air_BNB_test)

In [34]:
air_BNB_Train.columns

Index(['id', 'host_is_superhost', 'host_total_listings_count',
       'host_identity_verified', 'bedrooms', 'beds', 'price', 'maximum_nights',
       'minimum_maximum_nights', 'maximum_nights_avg_ntm',
       ...
       'property_type_Shared room in rental unit',
       'property_type_Shared room in residential home',
       'property_type_Tiny house', 'property_type_Tipi', 'property_type_Tower',
       'property_type_Yurt', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object', length=105)

In [35]:
from dash import Dash, html, dcc, Input, Output,dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
app = Dash()

In [36]:
layout = go.Layout(
    margin=go.layout.Margin(
        l=40,  # left margin
        r=40,  # right margin
        b=10,  # bottom margin
        t=35  # top margin
    ))

In [37]:
AirBnB_dropdown = dcc.Dropdown(options=air_BNB_Train['beds'].unique(),
                            value=3)

In [38]:
@app.callback(
    Output(component_id='price-graph', component_property='figure'),
    Input(component_id=AirBnB_dropdown, component_property='value')
)
def update_graph(selected_beds):
    filtered_AirBnB = air_BNB_Train[air_BNB_Train['beds'] == selected_beds]
    line_fig = px.line(filtered_AirBnB,
                       x='host_is_superhost', y='price',
                       color='maximum_nights',
                       title=f'AirBnB Prices for appartment with {selected_beds} beds and the correlation to maximum_nights',
                       width=600, height=400)
    return line_fig

In [39]:
def pie_graph():
    pieChart = dcc.Graph(
        figure=go.Figure(layout=layout).add_trace(go.Pie(
            labels=air_BNB_Train['bedrooms'],
            values=air_BNB_Train['host_total_listings_count'],
            marker=dict(colors=['#120303', '#300f0f', '#381b1b', '#4f2f2f', '#573f3f', '#695a5a', '#8a7d7d'],
                        line=dict(color='#ffffff', width=2)))).update_layout(title='Bedrooms Rate with accordence to host_total_listings_count ',
                                                                             plot_bgcolor='rgba(0,0,0,0)',
                                                                             showlegend=False),
        style={'width': 600,'height': 400, 'display': 'inline-block'})
    return pieChart

In [40]:
app.layout = html.Div([
    html.Div([
        html.Div([
            html.H1(
                children ='AirBnB Prices Dashboard'),
                AirBnB_dropdown
        ], className="six columns"),
         html.Div([
              dcc.Graph(id='price-graph',style = {'display' : 'inline-block'}),
                pie_graph()
         ], className="six columns"),
        html.Div([
            html.H1('DF', style={'text-align': 'center'}),
            dash_table.DataTable(air_BNB_Train.to_dict('records'), [{"name": i, "id": i} for i in air_BNB_Train.columns],
                 style_table={'overflowX': 'auto','height': '200px'}),
          ] , className="six columns"),
    ], className="row"),
    
])

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
    })

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
C:\ProgramData\Anaconda3\lib\site-packages\dash\resources.py:63: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

127.0.0.1 - - [11/Aug/2022 01:19:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 01:19:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 01:19:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 01:19:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 01:19:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 01:19:32] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -